In [1]:
#Imports
import resources
import pandas as pd
from blcalc.soilproperty import SoilProperty
from blcalc.material import LayerSoil
from blcalc.footing import Footing, FootingType, FootingData
from blcalc.assembly import Assembly
from blcalc.solver import Solver, Methods
import numpy as np

In [2]:
filepath = resources.secrets_dir() / "calc-merged.xlsx"

In [3]:
df = pd.read_excel(str(filepath))
df.columns

Index(['Depth', 'R.L of GWT', 'SPT(N)', 'N', 'Soil Type', 'Qu', 'Cu', 'phi',
       'Cb', 'Cs', 'Eh', 'Cr', 'gamma', 'w', 'G', 'Dry unit wt',
       'Saturated Unit weight', 'Vertical Effective Strees',
       'Total Effective Stress', 'CN', 'N60', 'FC', 'del(N1)60', '(N1)60CS',
       'CRR', 'Shear Reduction Factor', 'MSF', 'fake-Csigma', 'C-sigma',
       'fake-K-sigma', 'K-sigma', 'a/g', 'CSR', 'FS', 'Fz', 'Wz', 'Fz*wz*d',
       'lpi'],
      dtype='object')

In [4]:
fields_list=[] # (from,to)
starting = None
for i in range(len(df.index)):
    c=df['N60'][i]
    if(not pd.isnull(c) and starting is None):
        starting = i
    if (not starting is None) and pd.isnull(c):
        fields_list.append([starting, i])
        starting = None
fields_list.append([starting, len(df.index)])
print(len(fields_list), ' total borehole logs.')

104  total borehole logs.


In [5]:
locations_log = []
for idx,fl in enumerate(fields_list):
    i=fl[0]-2
    if not (pd.isnull(df['Depth'][i]) and pd.isnull(df['R.L of GWT'][i])):
        indx=[(df['Depth'][i],float(df['N'][i]),float(df['Soil Type'][i]))]
        locations_log.append(indx)
    locations_log[len(locations_log)-1].append(idx)
    fl.append(len(locations_log)-1)
print(len(locations_log), 'locations')

31 locations


In [6]:
for i in locations_log:
    print(i[0])

('Rastriya Banijya Bank: Thapathali, Kathmandu', 27.655862, 85.3064456)
('Building Complex: Anamnagar, Kathmandu', 27.6972818, 85.3274572)
('Building Complex: Bakhundol Lalitpur', 27.6830179, 85.3103347)
('Hindu vidhypeth: Balkumari, Lalitpur', 27.6711385, 85.3379345)
('DI Skin Health and Referral Center (P). Ltd.: Bansbari, Kathmandu', 27.7370376, 85.3335435)
('Building: Bhaisipati, Lalitpur', 27.648046, 85.2959632)
('Brihaspati Vidyasadan School: Gahanapokhari, Kathmandu', 27.7142534, 85.3295707)
('Green Hill City (P). Ltd. : Mulpani, Kathamndu ', 27.7157651, 85.386468)
('Building Site: Itachhe tol, Bhaktapur', 27.6728428, 85.4226077)
('Buddha Air (P). Ltd. : Jawalakhel, Lalitpur ', 27.6735721, 85.3118742)
('Tamrakar Samaj: Jawalakhel', 27.6747489, 85.312196)
('Tangal, Kathmandu', 27.7200932, 85.3262587)
('Harihar Bhavan, Lalitpur', 27.6804712, 85.3111097)
('Janamaitri Campus: Kuleswore, Kathmandu', 27.6917301, 85.290048)
('Amrit Science Campus: Lainchour, Kathmandu', 27.717759, 85.3

In [7]:
#get material info from table

def get_layer(row):
    layer = {}
    layer[SoilProperty.SPT_N] = df['SPT(N)'][row]
    layer[SoilProperty.depth] = df['Depth'][row]
    layer[SoilProperty.gamma] = df['gamma'][row]
    layer[SoilProperty.GI] = df['Soil Type'][row]
    if not pd.isnull(df['phi'][row]):
        layer[SoilProperty.phi] = df['phi'][row]
    if not pd.isnull(df['Qu'][row]):
        layer[SoilProperty.qu] = df['Qu'][row]
    if not pd.isnull(df['Cu'][row]):
        layer[SoilProperty.cu] = df['Cu'][row]
    layer[SoilProperty.water_per] = df['w'][row]
    layer[SoilProperty.G] = df['G'][row]
    layer[SoilProperty.FC] = df['FC'][row]
    return layer

def get_material(idx, isliquifaction=False):
    data=[]
    for i in range(fields_list[idx][0],fields_list[idx][1]):
        data.append(get_layer(i))
    water_depth = 0 
    if isliquifaction:
        water_depth = df['R.L of GWT'][fields_list[idx][0]]
    return LayerSoil(data, water_depth, idx)

In [8]:
#Get footing info
def get_footing(depth=1.5):
    footing = Footing()
    footing[FootingData.Type] = FootingType.Square
    footing[FootingData.Depth] = depth
    footing[FootingData.Width] = 2
    footing[FootingData.Length] = 2
    return footing

In [19]:
methods = []
for i in Methods:
    methods.append(i.value)

def get_result(ixx):
    res = []
    footing = get_footing()
    lay = get_material(ixx)
    lay2 = get_material(ixx,True)
    assembly = Assembly(footing, lay2)
    solver = Solver(assembly)
    lpi = solver.run([Methods.Liquifaction])[Methods.Liquifaction]
    for depth in [1.5,3,4.5]:
        footing = get_footing(depth)
        assembly = Assembly(footing, lay)
        solver = Solver(assembly)
        result = solver.run()
        result[Methods.Liquifaction] = lpi
        res.append(result)
    return res    
    
depths_results = [[],[],[]]

f=open('../secrets/results.tex', 'w')
#f.write('\\documentclass[12pt, a4paper]{scrreprt}\n')
#f.write('\\usepackage{times}\n')
#f.write('\\usepackage{multirow}\n')
#f.write('\\usepackage{lscape}\n')
#f.write('\\usepackage{tabularx}\n')
#f.write('\\begin{document}\n')
from pylatexenc.latexencode import unicode_to_latex as u2l

for i in locations_log:
    liquifactions_array = []
    shear_array = [[],[],[]]
    deflection_array = [[],[],[]]
    f.write('{\\large \\textbf{%s}}\\newline\n'%u2l(i[0][0]))
    bcount = 0
    for j in range(1,len(i)):
        bcount += 1
        result = get_result(i[j])
        liquifactions_array.append(result[0][Methods.Liquifaction])
        f.write('\\textbf{Borehole: %s}\n\\newline\n'%bcount)
        f.write('\\textbf{Shear}\\newline\n')
        f.write('\\begin{tabularx}{\\textwidth}{ | X | X | X | X | X | X | }\n')
        f.write('\\hline\n')
        f.write(' \\textbf{Depth} & \\textbf{Terzaghi}* & \\textbf{Meyerhof} & \\textbf{Hansen} & \\textbf{Vesic} & \\textbf{Plaxis} \\\\\n') 
        f.write('\\hline\n')
        f.write(' \\textbf{1.5} & %.2f & %.2f & %.2f & %.2f & %.2f \\\\\n'%(result[0][Methods.Terzaghi],result[0][Methods.Meyerhof],result[0][Methods.Hansen],result[0][Methods.Vesic],result[0][Methods.PlasixShear]))
        f.write(' \\textbf{3.0} & %.2f & %.2f & %.2f & %.2f & %.2f \\\\\n'%(result[1][Methods.Terzaghi],result[1][Methods.Meyerhof],result[1][Methods.Hansen],result[1][Methods.Vesic],result[1][Methods.PlasixShear]))
        f.write('\\hline\n')
        f.write('\\end{tabularx}\n\\hfill\\break\n')
        f.write('\\linebreak\n')
        f.write('\\textbf{Deflection}\\newline\n')
        f.write('\\begin{tabularx}{\\textwidth}{ | X | X | X | X | X | X | X | p{0.1\\linewidth} | }\n')
        f.write('\\hline\n')
        f.write(' \\textbf{Depth} & \\textbf{Bowels} & \\textbf{Teng} & \\textbf{Plaxis} & \\textbf{Peck} & \\textbf{Teng2} & \\textbf{Bowels2}* & \\textbf{Meyerhof} \\\\\n')
        f.write('\\hline\n')
        f.write(' \\textbf{1.5} & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f \\\\\n '%(result[0][Methods.Bowels],result[0][Methods.Teng],result[0][Methods.Plasix],result[0][Methods.Peck],result[0][Methods.TengDeflection],result[0][Methods.Bowels2],result[0][Methods.MeyerhofDeflection]))
        f.write(' \\textbf{3.0} & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f \\\\\n '%(result[1][Methods.Bowels],result[1][Methods.Teng],result[1][Methods.Plasix],result[1][Methods.Peck],result[1][Methods.TengDeflection],result[1][Methods.Bowels2],result[1][Methods.MeyerhofDeflection]))
        f.write(' \\textbf{4.5} & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f & %.2f \\\\\n '%(result[2][Methods.Bowels],result[2][Methods.Teng],result[2][Methods.Plasix],result[2][Methods.Peck],result[2][Methods.TengDeflection],result[2][Methods.Bowels2],result[2][Methods.MeyerhofDeflection]))
        f.write('\\hline\n')
        f.write('\\end{tabularx}\n\\hfill\\break\n')
        f.write('\\linebreak\n')
        f.write('\\textbf{LPI}: %.2f\n\\newline\n'%result[0][Methods.Liquifaction])
        f.write('\\newline\\break\n')
        for xx in range(3):
            #avg failed
            avg = (result[xx][Methods.Meyerhof]+result[xx][Methods.Hansen]+result[xx][Methods.Vesic])/3
            #shear_array[xx].append(result[xx][Methods.Meyerhof])
            #shear_array[xx].append(result[xx][Methods.Hansen])
            #shear_array[xx].append(result[xx][Methods.Vesic])
            shear_array[xx].append(avg)
            shear_array[xx].append(result[xx][Methods.PlasixShear])
            deflection_array[xx].append(result[xx][Methods.Bowels])
            deflection_array[xx].append(result[xx][Methods.Teng])
            deflection_array[xx].append(result[xx][Methods.Peck])
            deflection_array[xx].append(result[xx][Methods.TengDeflection])
            deflection_array[xx].append(result[xx][Methods.MeyerhofDeflection])
            deflection_array[xx].append(result[xx][Methods.Plasix])
    #We now compiled data for all locations
    liquifactions_avg = np.average(liquifactions_array)
    f.write('\\textbf{Summary:}\\newline\n')
    f.write('\\begin{tabularx}{\\textwidth}{ | X | X | X | X | X | X | }\n')
    f.write('\\hline\n')
    f.write(' \\textbf{Depth} & \\textbf{Shear} & \\textbf{Deflection} & \\textbf{LPI} & \\textbf{Latitude} & \\textbf{Longitude}\\\\\n') 
    f.write('\\hline\n')
    for xx,depthx in enumerate([1.5,3.0,4.5]):
        row = [i[0][0],i[0][1],i[0][2]]
        row.append(liquifactions_avg)
        row.append(np.quantile(shear_array[xx],0.5))
        row.append(np.quantile(deflection_array[xx],0.5))
        if xx==0:
            f.write(' \\textbf{%.1f} & %.2f & %.2f & \\multirow{3}{*}{%.2f} & \\multirow{3}{*}{%f} & \\multirow{3}{*}{%f} \\\\\n'%(depthx,np.quantile(shear_array[xx],0.5),np.quantile(deflection_array[xx],0.5),liquifactions_avg,i[0][1],i[0][2]))
        else:
            f.write(' \\textbf{%.1f} & %.2f & %.2f & & & \\\\\n'%(depthx,np.quantile(shear_array[xx],0.5),np.quantile(deflection_array[xx],0.5)))
        depths_results[xx].append(row)
    f.write('\\hline\n')
    f.write('\\end{tabularx}\n\\hfill\\break\n')
    #if j!=len(i)-1:
    f.write('\\linebreak\\newline\n')
    f.write('\\newline\n')
    #    f.write('\\hfill\\break\n')

f.write('\\begin{landscape}\n')
f.write('\\footnotesize\n')
for xx, depth in enumerate([1.5,3.0,4.5]):
    f.write('\\newpage\n')
    f.write('{\\large %.1fm}\\newline\n'% depth)
    f.write('\\newline\n')
    f.write('\\begin{tabular}{ | l | c | c | c | c | c | }\n')
    f.write('\\hline\n')
    f.write(' \\textbf{Location} & \\textbf{Latitude} & \\textbf{Longitude} & \\textbf{LPI} & \\textbf{Shear} & \\textbf{Deflection} \\\\\n')
    f.write('\\hline\n')
    for i in depths_results[xx]:
        f.write(' %s & %.5f & %.5f & %.2f & %.2f & %.2f \\\\\n'%(u2l(i[0]),i[1],i[2],i[3],i[4],i[5]))
    f.write('\\hline\n')
    f.write('\\end{tabular}\n')

f.write('\\end{landscape}\n')
#f.write('\\end{document}\n')
f.close()